# Votes (Divisions) in the Oireachtas: 01 - Data

An API is now available, see https://data.oireachtas.ie/ for documentation.

In [7]:
# Commonly useful notebook functions:
import sys
sys.path.insert(0,'../lib')
from common import *

In [ ]:
# Save json from https://api.oireachtas.ie/
# Max results available for paging through with limit & skip is 10,000 so splitting by years works best
def save_page(year, overwrite=False):
    save_page_year('divisions', year, overwrite)
    
# Entire dataset: 11475 as of 15-12-2017, starts at 1922
years = range(1922, datetime.now().year + 1)
process_with_progress(save_page, years)
print()

In [1]:
# To update after loading everything already: Grab the latest year and overwrite it:
save_page(datetime.now().year, True)

NameError: name 'save_page' is not defined

In [8]:
# Explore an example year of results (2002 has very few votes for some reason):
with lzma.open('data/year/2002.json.xz', 'rb') as f:
    preview = json.loads(f.read().decode('utf-8'))

In [9]:
RenderJSON(preview) # Preview all

In [10]:
RenderJSON(preview['results'][0]['division']) # Preview single vote

In [11]:
# Load Data from a single file and return dataframe
def divisions_df(fname):
    with lzma.open(fname, 'rb') as f:
        data = json.loads(f.read().decode('utf-8'))
        #print(fname, len(data['results']))
        
        # We don't care about the "head" part for now, just "results"
        records = json.dumps([r['division'] for r in data['results']])
        df = pd.read_json(records, orient='records', dtype=False)
        return df

In [13]:
# Process all the files and stick them together in a single dataframe:
files = glob('data/year/*.json.xz')

data_frames = process_with_progress(divisions_df, files)
df_raw = pd.concat(data_frames, axis=0, sort=True)

In [15]:
# Not all the columns are useful, we can transform and drop the redundant ones:
df_raw.sample(3)

,category,chamber,date,datetime,debate,house,isBill,outcome,subject,tallies,tellers,uri,voteId
52,Division,"{'showAs': 'Dáil Éireann', 'uri': '/ie/oireach...",1986-07-02,NaT,"{'showAs': 'National Lottery Bill, 1986: Commi...","{'showAs': '24th Dáil', 'houseCode': 'dail', '...",False,_,"{'showAs': 'Amendment put.', 'uri': None}","{'staonVotes': None, 'taVotes': {'showAs': 'Tá...","Tellers: Tá, Deputies F. O'Brien and Taylor; N...",/ie/oireachtas/division/house/dail/24/1986-07-...,vote_3
53,Division,"{'showAs': 'Seanad Éireann', 'uri': '/ie/oirea...",1993-10-13,NaT,{'showAs': 'Inadequate School Buildings: Motio...,"{'houseNo': '20', 'committeeCode': '', 'uri': ...",False,Carried,"{'showAs': 'Question put: ""That the motion, as...",{'taVotes': {'members': [{'member': {'showAs':...,"Tellers: Tá, Senators Mullooly and Magner; Níl...",/ie/oireachtas/division/house/seanad/20/1993-1...,vote_2
60,Division,"{'uri': '/ie/oireachtas/house/dail', 'showAs':...",1988-06-29,NaN,{'formats': {'xml': {'uri': '/akn/ie/debateRec...,"{'houseNo': '25', 'chamberType': 'house', 'hou...",False,Carried,"{'uri': None, 'showAs': 'Question put.'}","{'nilVotes': {'tally': 19, 'members': [{'membe...","Tellers: Tá, Deputies V. Brady and Browne; Níl...",/ie/oireachtas/division/house/dail/25/1988-06-...,vote_2


In [8]:
flatten_columns = ['chamber','debate','house','subject'] # deal with tallies later
df_raw = flatten_dataframe_columns(df_raw, flatten_columns)

# category is always "Division", isBill is always False, not used in "Divisions"
# debate.formats.pdf, subject.uri are always empty for this data
df_raw.drop(['category','isBill','debate.formats.pdf','subject.uri'], axis=1, inplace=True)

In [9]:
df_raw.sample(3)

,chamber.showAs,chamber.uri,debate.debateSection,debate.formats.xml.uri,debate.showAs,debate.uri,house.chamberType,house.committeeCode,house.houseCode,house.houseNo,house.showAs,house.uri,subject.showAs,date,datetime,outcome,tallies,tellers,uri,voteId
4,Seanad Éireann,/ie/oireachtas/house/seanad,dbsect_3,/akn/ie/debateRecord/seanad/1924-12-12/debate/...,"INTOXICATING LIQUOR BILL, 1924. - MESSAGE FROM...",/akn/ie/debateRecord/seanad/1924-12-12/debate/...,house,,seanad,1922,1922 Seanad,/ie/oireachtas/house/seanad/1922,"Question—""That the House do insist upon amendm...",1924-12-12,NaT,_,"{'staonVotes': None, 'nilVotes': {'showAs': 'N...",,/ie/oireachtas/division/house/seanad/1922/1924...,vote_1
31,Dáil Éireann,/ie/oireachtas/house/dail,dbsect_38,/akn/ie/debateRecord/dail/1943-03-25/debate/mu...,"Committee on Finance. - Army Pensions Bill, 19...",/akn/ie/debateRecord/dail/1943-03-25/debate/main,house,,dail,10,10th Dáil,/ie/oireachtas/house/dail/10,Question put.,1943-03-25,NaT,Carried,"{'nilVotes': {'tally': 21, 'members': [{'membe...",Tellers:—Tá: Deputies Smith and Brady; Níl: De...,/ie/oireachtas/division/house/dail/10/1943-03-...,vote_8
57,Dáil Éireann,/ie/oireachtas/house/dail,dbsect_14,/akn/ie/debateRecord/dail/1998-06-30/debate/mu...,Order of Business.,/akn/ie/debateRecord/dail/1998-06-30/debate/main,house,,dail,28,28th Dáil,/ie/oireachtas/house/dail/28,"Question put: ""That the proposals for dealing ...",1998-06-30,NaT,Carried,"{'nilVotes': {'tally': 59, 'members': [{'membe...","Tellers: Tá, Deputies S. Brennan and Power; Ní...",/ie/oireachtas/division/house/dail/28/1998-06-...,vote_1


In [10]:
describe_with_top_n(df_raw, exclude_columns=['tallies'], n=50)

----------

# chamber.showAs

###  Stats:

,chamber.showAs
count,12273
unique,74
top,Dáil Éireann
freq,7665
missing,0


###  Top N:

...

# voteId

###  Stats:

,voteId
count,12273
unique,199
top,vote_1
freq,5342
missing,0


###  Top N:

,voteId
vote_1,5342
vote_2,2719
vote_3,1430
vote_4,793
vote_5,464
vote_6,293
vote_7,172
vote_8,118
vote_9,83
vote_10,59


In [13]:
df_raw.to_pickle('divisions.p.xz', compression='xz') # gzip is faster, but xz makes much smaller files for git

In [14]:
import pandas_profiling

In [16]:
pandas_profiling.ProfileReport(df_raw)

Number of variables,21
Number of observations,12146
Total Missing (%),3.7%
Total size in memory,1.9 MiB
Average record size in memory,168.0 B
Numeric,1
Categorical,17
Boolean,0
Date,2
Text (Unique),0
Rejected,0
